#Monitoring and Logging with Internal Components

In [1]:
import logging
import sys

# Create a logger
logger = logging.getLogger('my_logger')
logger.setLevel(logging.DEBUG)  # Set logger level to capture all levels of messages

# Create a handler for notebook output
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)  # Set handler level to capture INFO and above

# Define a formatter
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

# Add the handler to the logger
logger.addHandler(handler)

# Example usage
def my_function():
    logger.info("This is an informational message.")
    logger.warning("This is a warning message.")
    logger.error("This is an error message.")

my_function()

2024-05-10 23:45:31,567 - INFO - This is an informational message.


INFO:my_logger:This is an informational message.


2024-05-10 23:45:31,580 - WARNING - This is a warning message.


2024-05-10 23:45:31,584 - ERROR - This is an error message.


ERROR:my_logger:This is an error message.


#Loguru

In [2]:
!pip install loguru

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 1.3 MB/s eta 0:00:00


In [4]:
from loguru import logger

# Define custom format with complex representation
log_format = (
    "<green>{time:YYYY-MM-DD HH:mm:ss}</green> | "
    "<level>{level: <8}</level> | "
    "<cyan>{name}</cyan>:<cyan>{function}</cyan>:<cyan>{line}</cyan> - "
    "<level>{message}</level>"
)

# Configure Loguru with the custom format and output to console and file
logger.add("app.log", rotation="500 MB", format=log_format)  # Output to file with custom format
logger.add(lambda msg: print(msg, end=""), colorize=True, format=log_format)  # Output to console with custom format

# Sample function that logs messages at different levels
def perform_tasks():
    logger.info("Starting tasks...")
    try:
        # Simulate some tasks
        for i in range(1, 6):
            logger.trace(f"Task {i}: Processing...")
            # Perform some task that might generate warnings or errors
            if i == 3:
                logger.warning("Task 3 encountered an issue!")
            elif i == 5:
                raise Exception("Task 5 failed due to an unexpected error!")
            logger.debug(f"Task {i}: Completed successfully.")
        logger.success("All tasks completed successfully!")
    except Exception as e:
        logger.error(f"Error occurred during tasks: {e}", exc_info=True)  # Include exception info

if __name__ == "__main__":
    # Perform tasks and log messages
    logger.info("Application starting...")
    perform_tasks()
    logger.info("Application finished.")


2024-05-10 23:51:14.399 | INFO     | __main__:<cell line: 32>:34 - Application starting...
2024-05-10 23:51:14.403 | INFO     | __main__:perform_tasks:17 - Starting tasks...
2024-05-10 23:51:14.407 | DEBUG    | __main__:perform_tasks:27 - Task 1: Completed successfully.
2024-05-10 23:51:14.410 | DEBUG    | __main__:perform_tasks:27 - Task 2: Completed successfully.
2024-05-10 23:51:14.414 | WARNING  | __main__:perform_tasks:24 - Task 3 encountered an issue!
2024-05-10 23:51:14.417 | DEBUG    | __main__:perform_tasks:27 - Task 3: Completed successfully.
2024-05-10 23:51:14.419 | DEBUG    | __main__:perform_tasks:27 - Task 4: Completed successfully.
2024-05-10 23:51:14.421 | ERROR    | __main__:perform_tasks:30 - Error occurred during tasks: Task 5 failed due to an unexpected error!
2024-05-10 23:51:14.422 | INFO     | __main__:<cell line: 32>:36 - Application finished.


2024-05-10 23:51:14 | INFO     | __main__:<cell line: 32>:34 - Application starting...
2024-05-10 23:51:14 | INFO     | __main__:perform_tasks:17 - Starting tasks...
2024-05-10 23:51:14 | DEBUG    | __main__:perform_tasks:27 - Task 1: Completed successfully.
2024-05-10 23:51:14 | DEBUG    | __main__:perform_tasks:27 - Task 2: Completed successfully.
2024-05-10 23:51:14 | WARNING  | __main__:perform_tasks:24 - Task 3 encountered an issue!
2024-05-10 23:51:14 | DEBUG    | __main__:perform_tasks:27 - Task 3: Completed successfully.
2024-05-10 23:51:14 | DEBUG    | __main__:perform_tasks:27 - Task 4: Completed successfully.
2024-05-10 23:51:14 | ERROR    | __main__:perform_tasks:30 - Error occurred during tasks: Task 5 failed due to an unexpected error!
2024-05-10 23:51:14 | INFO     | __main__:<cell line: 32>:36 - Application finished.


#Structlog

In [5]:
!pip install structlog

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 2.0 MB/s eta 0:00:00


In [6]:
import sys
import structlog
from structlog.processors import JSONRenderer, TimeStamper
from structlog.stdlib import LoggerFactory
import logging

# Configure structlog to use a custom logger factory
structlog.configure(
    logger_factory=LoggerFactory(),
    processors=[
        TimeStamper(fmt="%Y-%m-%d %H:%M:%S"),
        JSONRenderer()
    ]
)

# Create a logger that will output to both console and file
def create_logger():
    handlers = [
        logging.StreamHandler(sys.stdout),  # Output to console
        logging.FileHandler("app.log")     # Output to file
    ]
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)
    for handler in handlers:
        logger.addHandler(handler)
    return structlog.wrap_logger(logger, processors=structlog.get_config()["processors"])

# Sample function that logs messages at different levels
def perform_tasks(logger):
    logger.info("Starting tasks...")
    try:
        # Simulate some tasks
        for i in range(1, 6):
            logger.debug("Processing task", task_id=i)
            # Perform some task that might generate warnings or errors
            if i == 3:
                logger.warning("Task 3 encountered an issue!")
            elif i == 5:
                raise Exception("Task 5 failed due to an unexpected error!")
            logger.debug("Task completed successfully", task_id=i)
        logger.info("All tasks completed successfully!")
    except Exception as e:
        logger.error("Error occurred during tasks", error=str(e))

if __name__ == "__main__":
    # Create logger with custom formatting and multiple outputs
    logger = create_logger()

    # Perform tasks and log messages
    logger.info("Application starting...")
    perform_tasks(logger)
    logger.info("Application finished.")

{"event": "Application starting...", "timestamp": "2024-05-10 23:57:21"}


INFO:__main__:{"event": "Application starting...", "timestamp": "2024-05-10 23:57:21"}


{"event": "Starting tasks...", "timestamp": "2024-05-10 23:57:21"}


INFO:__main__:{"event": "Starting tasks...", "timestamp": "2024-05-10 23:57:21"}


{"event": "Task 3 encountered an issue!", "timestamp": "2024-05-10 23:57:21"}


{"error": "Task 5 failed due to an unexpected error!", "event": "Error occurred during tasks", "timestamp": "2024-05-10 23:57:21"}


ERROR:__main__:{"error": "Task 5 failed due to an unexpected error!", "event": "Error occurred during tasks", "timestamp": "2024-05-10 23:57:21"}


{"event": "Application finished.", "timestamp": "2024-05-10 23:57:21"}


INFO:__main__:{"event": "Application finished.", "timestamp": "2024-05-10 23:57:21"}


#Logbook

In [7]:
!pip install logbook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.0/491.0 kB 6.5 MB/s eta 0:00:00


In [8]:
import sys
import logbook
from logbook import Logger, FileHandler, StreamHandler

# Define custom log format
log_format = "{record.time:%Y-%m-%d %H:%M:%S} | {record.level_name} | {record.channel}:{record.func_name}:{record.lineno} - {record.message}"

# Create logger and configure handlers
logger = Logger("MyApp")
file_handler = FileHandler("app.log", level=logbook.INFO, format_string=log_format)
stream_handler = StreamHandler(sys.stdout, level=logbook.INFO, format_string=log_format)

# Apply handlers to the logger
logger.handlers.append(file_handler)
logger.handlers.append(stream_handler)

# Sample function that performs logging
def perform_tasks():
    logger.info("Starting tasks...")
    try:
        # Simulate some tasks
        for i in range(1, 6):
            logger.debug(f"Processing task {i}")
            # Perform some task that might generate warnings or errors
            if i == 3:
                logger.warn("Task 3 encountered an issue!")
            elif i == 5:
                raise Exception("Task 5 failed due to an unexpected error!")
            logger.debug(f"Task {i} completed successfully.")
        logger.info("All tasks completed successfully!")
    except Exception as e:
        logger.error(f"Error occurred during tasks: {e}", exc_info=True)

if __name__ == "__main__":
    # Perform tasks and log messages
    logger.info("Application starting...")
    perform_tasks()
    logger.info("Application finished.")


#Eliot

In [9]:
!pip install eliot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.1/113.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.3/247.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.7/191.7 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 18.6 MB/s eta 0:00:00


In [11]:
from eliot import start_action, to_file
import sys

# Configure Eliot to log to a file
to_file(open("app.log", "w"))

# Sample function that performs tasks and logs actions
def perform_tasks():
    with start_action(action_type="perform_tasks"):
        print("Starting tasks...")
        try:
            # Simulate some tasks
            for i in range(1, 6):
                with start_action(action_type="process_task", task_id=i):
                    print(f"Processing task {i}")

                    # Perform some task that might generate warnings or errors
                    if i == 3:
                        raise ValueError("Task 3 encountered an issue!")
                    elif i == 5:
                        raise Exception("Task 5 failed due to an unexpected error!")

                    print(f"Task {i} completed successfully.")

            print("All tasks completed successfully!")

        except Exception as e:
            print(f"Error occurred during tasks: {e}")

# Main part of the script
if __name__ == "__main__":
    print("Application starting...")
    perform_tasks()
    print("Application finished.")


Application starting...
Starting tasks...
Processing task 1
Task 1 completed successfully.
Processing task 2
Task 2 completed successfully.
Processing task 3
Error occurred during tasks: Task 3 encountered an issue!
Application finished.
